In [1]:
from functools import reduce
from pyspark.sql.functions import col, lit, when
from graphframes import *

In [2]:
%sh curl -O 'https://s3.eu-central-1.amazonaws.com/meetupsocialnetwork/groups_nodefile.csv'

% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
 3 1329k 3 51865 0 0 41095 0 0:00:33 0:00:01 0:00:32 41097
100 1329k 100 1329k 0 0 706k 0 0:00:01 0:00:01 --:--:-- 705k

In [3]:
%sh curl -O 'https://s3.eu-central-1.amazonaws.com/meetupsocialnetwork/group_nw_.csv'

% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
 0 1277M 0 51858 0 0 41306 0 9:00:28 0:00:01 9:00:27 41288
 0 1277M 0 4619k 0 0 2119k 0 0:10:17 0:00:02 0:10:15 2118k
 1 1277M 1 21.0M 0 0 6542k 0 0:03:19 0:00:03 0:03:16 6540k
 2 1277M 2 35.0M 0 0 8462k 0 0:02:34 0:00:04 0:02:30 8463k
 3 1277M 3 49.4M 0 0 9750k 0 0:02:14 0:00:05 0:02:09 9.9M
 5 1277M 5 66.1M 0 0 10.5M 0 0:02:01 0:00:06 0:01:55 13.1M
 6 1277M 6 80.6M 0 0 11.1M 0 0:01:54 0:00:07 0:01:47 15.0M
 7 1277M 7 94.9M 0 0 11.5M 0 0:01:50 0:00:08 0:01:42 15.0M
 8 1277M 8 111M 0 0 11.9M 0 0:01:46 0:00:09 0:01:37 15.0M
 9 1277M 9 124M 0 0 12.1M 0 0:01:44 0:00:10 0:01:34 14.9M
 10 1277M 10 139M 0 0 12.4M 0 0:01:42 0:00:11 0:01:31 14.8M
 12 1277M 12 155M 0 0 12.7M 0 0:01:39 0:00:12 0:01:27 15.2M
 13 1277M 13 170M 0 0 12.8M 0 0:01:39 0:00:13 0:01:26 14.8M
 14 1277M 14 184M 0 0 12.9M 0 0:01:38 0:00:14 0:01:24 14.8M
 15 1277M 15 199M 0 0 13.1M 0 0:01:37 0:00:15 0:01:22 15.0M
 16 1277M 16 215M 0 0 13.2M 0 0:01:36 0:00:16 0:01:20 15.1M
 18 1277M 18 230M 0 0 13.3M 0 0:01:35 0:00:17 0:01:18 14.7M
 19 1277M 19 244M 0 0 13.4M 0 0:01:34 0:00:18 0:01:16 15.2M
 20 1277M 20 261M 0 0 13.6M 0 0:01:33 0:00:19 0:01:14 15.5M
 21 1277M 21 276M 0 0 13.6M 0 0:01:33 0:00:20 0:01:13 15.1M
 22 1277M 22 290M 0 0 13.7M 0 0:01:33 0:00:21 0:01:12 15.1M
 23 1277M 23 306M 0 0 13.8M 0 0:01:32 0:00:22 0:01:10 15.3M
 25 1277M 25 322M 0 0 13.8M 0 0:01:32 0:00:23 0:01:09 15.1M
 26 1277M 26 336M 0 0 13.8M 0 0:01:31 0:00:24 0:01:07 14.8M
 27 1277M 27 352M 0 0 13.9M 0 0:01:31 0:00:25 0:01:06 15.3M
 28 1277M 28 367M 0 0 14.0M 0 0:01:31 0:00:26 0:01:05 15.2M
 29 1277M 29 382M 0 0 14.0M 0 0:01:30 0:00:27 0:01:03 15.0M
 31 1277M 31 397M 0 0 14.1M 0 0:01:30 0:00:28 0:01:02 15.3M
 32 1277M 32 413M 0 0 14.1M 0 0:01:30 0:00:29 0:01:01 15.2M
 33 1277M 33 428M 0 0 14.1M 0 0:01:30 0:00:30 0:01:00 15.0M
 34 1277M 34 442M 0 0 14.2M 0 0:01:29 0:00:31 0:00:58 15.3M
 35 1277M 35 459M 0 0 14.2M 0 0:01:29 0:00:32 0:00:57 15.2M
 37 1277M 37 473M 0 0 14.2M 0 0:01:29 0:00:33 0:00:56 15.1M
 38 1277M 38 488M 0 0 14.2M 0 0:01:29 0:00:34 0:00:55 15.2M
 39 1277M 39 505M 0 0 14.3M 0 0:01:29 0:00:35 0:00:54 15.2M
 40 1277M 40 519M 0 0 14.3M 0 0:01:29 0:00:36 0:00:53 15.1M
 41 1277M 41 534M 0 0 14.3M 0 0:01:29 0:00:37 0:00:52 15.1M
 43 1277M 43 549M 0 0 14.3M 0 0:01:28 0:00:38 0:00:50 15.2M
 44 1277M 44 565M 0 0 14.4M 0 0:01:28 0:00:39 0:00:49 15.1M
 45 1277M 45 579M 0 0 14.4M 0 0:01:28 0:00:40 0:00:48 15.1M
 46 1277M 46 595M 0 0 14.4M 0 0:01:28 0:00:41 0:00:47 15.2M
 47 1277M 47 611M 0 0 14.4M 0 0:01:28 0:00:42 0:00:46 15.2M
 48 1277M 48 625M 0 0 14.4M 0 0:01:28 0:00:43 0:00:45 15.1M
 50 1277M 50 641M 0 0 14.5M 0 0:01:28 0:00:44 0:00:44 15.3M
 51 1277M 51 657M 0 0 14.5M 0 0:01:27 0:00:45 0:00:42 15.2M
 52 1277M 52 671M 0 0 14.5M 0 0:01:27 0:00:46 0:00:41 15.1M
 53 1277M 53 686M 0 0 14.5M 0 0:01:27 0:00:47 0:00:40 15.2M
 54 1277M 54 701M 0 0 14.5M 0 0:01:27 0:00:48 0:00:39 15.2M
 56 1277M 56 717M 0 0 14.5M 0 0:01:27 0:00:49 0:00:38 15.1M
 57 1277M 57 731M 0 0 14.5M 0 0:01:27 0:00:50 0:00:37 15.1M
 58 1277M 58 747M 0 0 14.5M 0 0:01:27 0:00:51 0:00:36 15.2M
 59 1277M 59 763M 0 0 14.6M 0 0:01:27 0:00:52 0:00:35 15.2M
 60 1277M 60 777M 0 0 14.6M 0 0:01:27 0:00:53 0:00:34 15.1M
 62 1277M 62 793M 0 0 14.6M 0 0:01:27 0:00:54 0:00:33 15.2M
 63 1277M 63 809M 0 0 14.6M 0 0:01:27 0:00:55 0:00:32 15.2M
 64 1277M 64 823M 0 0 14.6M 0 0:01:27 0:00:56 0:00:31 15.1M
 65 1277M 65 839M 0 0 14.6M 0 0:01:27 0:00:57 0:00:30 15.2M
 66 1277M 66 854M 0 0 14.6M 0 0:01:26 0:00:58 0:00:28 15.5M
 68 1277M 68 869M 0 0 14.6M 0 0:01:27 0:00:59 0:00:28 15.1M
 69 1277M 69 884M 0 0 14.7M 0 0:01:26 0:01:00 0:00:26 15.4M
 70 1277M 70 899M 0 0 14.7M 0 0:01:26 0:01:01 0:00:25 15.4M
 71 1277M 71 915M 0 0 14.7M 0 0:01:26 0:01:02 0:00:24 15.1M
 72 1277M 72 929M 0 0 14.7M 0 0:01:26 0:01:03 0:00:23 14.9M
 73 1277M 73 945M 0 0 14.7M 0 0:01:26 0

In [4]:
%sh curl -O 'https://s3.eu-central-1.amazonaws.com/meetupsocialnetwork/venues_nodefile.csv'

% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
100 243 0 243 0 0 285 0 --:--:-- --:--:-- --:--:-- 285

In [5]:
%sh curl -O 'https://s3.eu-central-1.amazonaws.com/meetupsocialnetwork/venues_edgefile.csv'

% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
100 243 0 243 0 0 285 0 --:--:-- --:--:-- --:--:-- 285

In [6]:
%fs ls "file:/databricks/driver"

path,name,size
file:/databricks/driver/conf/,conf/,4096
file:/databricks/driver/logs/,logs/,4096
file:/databricks/driver/derby.log,derby.log,731
file:/databricks/driver/venues_edgefile.csv,venues_edgefile.csv,243
file:/databricks/driver/eventlogs/,eventlogs/,4096
file:/databricks/driver/group_nw_.csv,group_nw_.csv,1339503326
file:/databricks/driver/groups_nodefile.csv,groups_nodefile.csv,1361227


In [7]:
# Read nodes and edges for group network
group_nodes = spark.read.csv('file:/databricks/driver/groups_nodefile.csv',inferSchema=True, header =True)
group_edges = spark.read.csv('file:/databricks/driver/group_nw_.csv',inferSchema=True, header =True)

print(group_nodes.count()) #16330 nodes
print(group_edges.count()) # 13079090 edges

16330
13079090

In [8]:
group_nodes.printSchema()

root
-- NodeId: integer (nullable = true)
-- GroupName: string (nullable = true)
-- CategoryID: integer (nullable = true)
-- # Members: integer (nullable = true)
-- City: string (nullable = true)
-- Date Created: timestamp (nullable = true)
-- Rating: double (nullable = true)

In [9]:
group_edges.printSchema()

root
-- _c0: integer (nullable = true)
-- EdgeID: integer (nullable = true)
-- group_from: integer (nullable = true)
-- group_to: integer (nullable = true)
-- weight: integer (nullable = true)
-- comm_members: string (nullable = true)

In [10]:
# Change colums name to construct the graph
oldColumns = group_nodes.schema.names
newColumns = ["id", "name", "category_id", "num_members", "city", "date_created", "rating"]

group_nodes = reduce(lambda data, idx: data.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), group_nodes)
group_nodes.printSchema()

root
-- id: integer (nullable = true)
-- name: string (nullable = true)
-- category_id: integer (nullable = true)
-- num_members: integer (nullable = true)
-- city: string (nullable = true)
-- date_created: timestamp (nullable = true)
-- rating: double (nullable = true)

In [11]:
# Change column names to construct the graph
oldColumns = group_edges.schema.names
newColumns = ["id", "edge_id", "src", "dst", "weight", "common_members"]

group_edges = reduce(lambda data, idx: data.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), group_edges)
group_edges.printSchema()

root
-- id: integer (nullable = true)
-- edge_id: integer (nullable = true)
-- src: integer (nullable = true)
-- dst: integer (nullable = true)
-- weight: integer (nullable = true)
-- common_members: string (nullable = true)

In [12]:
# Reduce number of edges
from pyspark.sql.functions import col
group_edges_reduced = group_edges.filter(col("weight") >= 5)


In [13]:
print(group_edges.count())
print(group_edges_reduced.count())

13079090
3160525

In [14]:
# Create the graph
g = GraphFrame(group_nodes, group_edges_reduced)

In [15]:
smaller = g.vertices.groupBy().min("num_members")
display(smaller)

min(num_members)
1


In [16]:
bigger = g.vertices.groupBy().max("num_members")
display(bigger)

max(num_members)
31215


In [17]:
display(g.vertices)

id,name,category_id,num_members,city,date_created,rating
6388,Alternative Health NYC,14,1440,New York,2002-11-21T16:50:46.000+0000,4.39
6510,Alternative Energy Meetup,4,969,New York,2003-05-20T14:48:54.000+0000,4.31
8458,NYC Animal Rights,26,2930,New York,2004-03-27T09:55:41.000+0000,4.84
8940,The New York City Anime Group,29,5080,New York,2002-11-16T04:49:16.000+0000,4.46
10104,NYC Pit Bull Group,26,2097,New York,2003-10-22T21:39:49.000+0000,4.09
10359,NYC International Arabic Language & Culture Club,16,1171,New York,2003-05-22T14:19:52.000+0000,4.58
12111,The New York City American Sign Language Meetup Group,16,140,New York,2002-10-08T17:22:28.000+0000,4.46
12542,The New York City Astrology Meetup,22,1738,New York,2002-10-08T17:22:23.000+0000,4.59
12907,New York City Atheists Meetings,28,2454,New York,2002-10-30T03:09:34.000+0000,4.63
14573,The New York City Ayn Rand Group,13,557,New York,2003-04-14T17:20:41.000+0000,4.56


In [18]:
display(g.edges)

id edge_id src dst weight common_members 0 0 490552 1474611 57 [3, 1418715, 1671282, 2148610, 2319427, 2709298, 2741328, 3169887, 3226548, 3592226, 3775564, 3890522, 4297339, 4331241, 5029229, 5620776, 5789458, 7607137, 8468628, 8693413, 9245645, 9308780, 9504442, 11108096, 11825789, 12093969, 12313224, 13861471, 30204882, 45304242, 45684522, 91319762, 104440312, 115507992, 142427502, 181895502, 183329781, 183757846, 184755010, 184942179, 185540240, 185711418, 185739525, 186244961, 186625433, 186884321, 187850939, 190974951, 192514972, 194186667, 200829702, 201854997, 201940363, 203147732, 208119525, 211275878, 213263083] 1 1 490552 1490492 66 [3, 1418715, 1584644, 2148610, 2709298, 3300861, 3592226, 3826156, 3890522, 4023165, 4097057, 4212790, 4340948, 4526750, 5029229, 5620776, 5789458, 5954245, 6431237, 6705140, 6958909, 7217725, 8312903, 8371582, 9131581, 9245645, 9306675, 9504442, 10496458, 10632989, 10750025, 10829602, 11041705, 11108096, 12093969, 13276555, 13566300, 13824655, 13918962, 13975863, 14221662, 14451551, 26187642, 31492942, 37209012, 45684522, 57566832, 64537082, 91318072, 100644052, 112736122, 135580162, 181895502, 182649499, 183329781, 184202937, 184221078, 184348918, 184755010, 185083341, 185963137, 186625433, 186669418, 188796105, 190974951, 191633392] 2 2 490552 1515830 167 [3, 1418715, 1597206, 1671282, 2148610, 2220252, 2496160, 2877642, 2972671, 3169887, 3181426, 3226548, 3300861, 3342013, 3592226, 3775564, 3810896, 3826156, 3925169, 4097057, 4212790, 4331241, 4340948, 5029229, 5620776, 5789458, 5826019, 6285042, 6285784, 6295534, 6876644, 6958909, 7217725, 7246838, 7426060, 7472311, 8371582, 8468628, 8693413, 9245645, 9504442, 10366201, 10632989, 11041705, 11192984, 11825789, 11926529, 12093969, 12566946, 13051582, 13329930, 13390033, 13861471, 13918962, 14221662, 14242793, 14451551, 21146231, 30204882, 37756082, 45304242, 57566832, 66214572, 77107442, 96267532, 100644052, 104440312, 112736122, 115507992, 116018732, 116761632, 124786812, 124833932, 132035062, 159155142, 177625822, 180210262, 181895502, 182649499, 183329781, 183417875, 183679635, 184221078, 184295112, 184348918, 184605126, 185343927, 185516598, 185675499, 185711452, 186233159, 186625433, 186669418, 186957479, 187405748, 187850939, 188532151, 189000563, 189021660, 189058705, 189462531, 190036811, 190974951, 191633392, 192514972, 193783737, 193918455, 195103922, 195788678, 197125844, 197417264, 198393427, 198571871, 198611759, 199139615, 201257244, 202686256, 203147732, 205510514, 206337168, 206947318, 207125360, 208119525, 210773539, 210791034, 210904396, 211275878, 211413603, 212486018, 212973561, 213794549, 213970819, 214221841, 214600680, 215095050, 215253724, 218010154, 219445363, 219898810, 220103001, 220288124, 221223822, 222824503, 223326608, 223547550, 223787449, 225382547, 226495961, 227011247, 227765943, 227816999, 228888786, 228966438, 228994890, 229908274, 230493176, 230712669, 230879378, 230982694, 231039234, 231668232, 231948039, 232536441, 232985942, 235064725, 238687841, 239669257] 3 3 490552 1574965 46 [3, 1418715, 1597206, 1671282, 2148610, 2496160, 3169887, 3226548, 3483447, 3592226, 3826156, 4097057, 4297339, 4340948, 4526750, 5029229, 5620776, 5789458, 6958909, 7472311, 8468628, 8693413, 9245645, 10289634, 11041705, 11300185, 11825789, 12093969, 13861471, 14221662, 21146231, 30204882, 31492942, 37756082, 45304242, 45684522, 51770172, 67119482, 91318072, 100644052, 135580162, 181895502, 189000563, 190974951, 216349950, 228589723] 4 4 490552 1669000 62 [3, 1584644, 1597206, 2684823, 2741328, 3181426, 3361856, 3592226, 3775564, 4273357, 5166001, 5620776, 7472311, 8772513, 9245645, 10750025, 12093969, 12313224, 12559818, 20100751, 57566832, 79068232, 94148492, 95445572, 100644052, 120822842, 124833932, 132979882, 135334272, 181895502, 183329781, 183530787, 185343927, 186002668, 187850939, 190974951, 194186667, 198449617, 198611759, 201233389, 201257244, 201915129, 202686256, 204532506, 208105025, 208119525, 208582566,

In [19]:
display(g.degrees)

id,degree
21471196,378
1748515,1635
37146,412
18693412,1091
3979102,1935
807758,807
6892592,1537
1539044,689
5087172,1565
1483633,1284


In [20]:
from __future__ import print_function

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.datasets.samples_generator import make_blobs
from pyspark import SparkContext
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SQLContext

In [21]:
df = group_nodes
# Add degree property to each node
df = (g.degrees).join(df, 'id')
display(df)

id,degree,name,category_id,num_members,city,date_created,rating
21471196,378,The Society for Constitutional Protection,13,393,New York,2016-12-05T22:02:59.000+0000,4.81
1748515,1635,LispNYC,34,1548,New York,2010-12-09T14:23:53.000+0000,4.75
37146,412,The Chicago Comic Book Meetup Group,29,731,Chicago,2003-02-02T17:47:47.000+0000,4.6
18693412,1091,Goethe-Institut,16,1124,New York,2015-06-23T15:46:42.000+0000,4.63
3979102,1935,Angular-SF,34,4093,San Francisco,2012-05-31T21:15:29.000+0000,4.64
807758,807,The Miramar Ski Club,23,1063,New York,2007-11-09T18:25:56.000+0000,4.8
6892592,1537,Singles Travel Together,23,6437,Chicago,2013-01-28T15:31:55.000+0000,0.0
1539044,689,Business Intelligence User Group of Chicago,34,793,Chicago,2009-10-12T19:02:57.000+0000,4.88
5087172,1565,New York Haskell Users Group,34,1830,New York,2012-09-24T21:26:02.000+0000,4.84
1483633,1284,Clojure NYC,34,1457,New York,2009-07-03T18:50:28.000+0000,4.57


In [22]:
# Map city to an integer value
cities_mapping = {
"New York": 1,
"Chicago": 2,
"San Francisco": 3,
"South San Francisco": 4,
"West Chicago": 5,
"Chicago Ridge": 6,
"Chicago Heights": 7,
"West New York": 8,
"North Chicago": 9
}
def f(x): 
    return cities_mapping[x]

df = df.withColumn("city", when(col("city")=='New York', 1).when(col("city")=='Chicago', 2)
              .when(col("city")=='San Francisco', 3).when(col("city")=='South San Francisco', 4)
              .when(col("city")=='West Chicago', 5).when(col("city")=='Chicago Ridge', 6)
              .when(col("city")=='Chicago Heights', 7).when(col("city")=='West New York', 8)
              .when(col("city")=='North Chicago', 9).otherwise(0))

df.show()

+--------+------+--------------------+-----------+-----------+----+-------------------+------+
 id|degree| name|category_id|num_members|city| date_created|rating|
+--------+------+--------------------+-----------+-----------+----+-------------------+------+
21471196| 378|The Society for C...| 13| 393| 1|2016-12-05 22:02:59| 4.81|
 1748515| 1635| LispNYC| 34| 1548| 1|2010-12-09 14:23:53| 4.75|
 37146| 412|The Chicago Comic...| 29| 731| 2|2003-02-02 17:47:47| 4.6|
18693412| 1091| Goethe-Institut| 16| 1124| 1|2015-06-23 15:46:42| 4.63|
 3979102| 1935| Angular-SF| 34| 4093| 3|2012-05-31 21:15:29| 4.64|
 807758| 807|The Miramar Ski Club| 23| 1063| 1|2007-11-09 18:25:56| 4.8|
 6892592| 1537|Singles Travel To...| 23| 6437| 2|2013-01-28 15:31:55| 0.0|
 1539044| 689|Business Intellig...| 34| 793| 2|2009-10-12 19:02:57| 4.88|
 5087172| 1565|New York Haskell ...| 34| 1830| 1|2012-09-24 21:26:02| 4.84|
 1483633| 1284| Clojure NYC| 34| 1457| 1|2009-07-03 18:50:28| 4.57|
19308825| 1655|Experiencing Cult...| 10| 2151| 1|2016-01-09 16:13:20| 5.0|
13494372| 1188|SALSA NEW YORK Cl...| 5| 2115| 1|2014-03-23 01:16:47| 5.0|
 1460361| 1112|Heal the Loss of ...| 30| 1139| 1|2009-05-22 15:07:37| 4.44|
14818872| 286|Weekly Tuesday Ev...| 14| 229| 3|2014-06-02 04:22:19| 5.0|
22034166| 42|Soular Charge S...| 22| 46| 2|2017-01-22 20:08:36| 0.0|
 687322| 1513|Business is Art &...| 15| 1440| 1|2007-09-16 01:06:38| 4.73|
18239213| 856|Polymath Conversa...| 6| 505| 1|2014-12-08 05:20:07| 4.63|
19046161| 1248|New York Songwrit...| 21| 1765| 1|2015-10-20 22:33:10| 4.86|
19134120| 206|NYC Women Driving...| 2| 197| 1|2015-11-18 17:10:15| 4.83|
 8392492| 2206|Manhattan Lecture...| 6| 1511| 1|2013-05-07 23:49:56| 4.85|
+--------+------+--------------------+-----------+-----------+----+-------------------+------+
only showing top 20 rows

In [23]:
max_src = g.edges.groupBy("src").max("weight")
min_src = g.edges.groupBy("src").min("weight")
avg_src = g.edges.groupBy("src").avg("weight")

max_dst = g.edges.groupBy("dst").max("weight")
min_dst = g.edges.groupBy("dst").min("weight")
avg_dst = g.edges.groupBy("dst").avg("weight")

In [24]:
# Change column names to construct the graph
oldColumns = max_src.schema.names
newColumns = ["id", "max"]

max_src = reduce(lambda data, idx: data.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), max_src)


oldColumns = max_dst.schema.names
newColumns = ["id", "max",]

max_dst = reduce(lambda data, idx: data.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), max_dst)


oldColumns = min_src.schema.names
newColumns = ["id", "min"]

min_src = reduce(lambda data, idx: data.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), min_src)


oldColumns = min_dst.schema.names
newColumns = ["id", "min" ]

min_dst = reduce(lambda data, idx: data.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), min_dst)

oldColumns = avg_src.schema.names
newColumns = ["id", "avg"]

avg_src = reduce(lambda data, idx: data.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), avg_src)


oldColumns = avg_dst.schema.names
newColumns = ["id", "avg" ]

avg_dst = reduce(lambda data, idx: data.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), avg_dst)



In [25]:
max_weights = max_src.union(max_dst)
min_weights = min_src.union(min_dst)
avg_weights = avg_src.union(avg_dst)

In [26]:
g.degrees.schema

Out[ 20 ]: StructType(List(StructField(id,IntegerType,true),StructField(degree,IntegerType,false)))

In [27]:
print(max_weights.count())
print(min_weights.count())
print(avg_weights.count())

22297
22297
22297

In [28]:
max_w=max_weights.groupBy("id").sum("max")

In [29]:
min_w=min_weights.groupBy("id").sum("min")
avg_w=avg_weights.groupBy("id").sum("avg")

In [30]:
display(min_w)

+--------+--------+
 id|sum(min)|
+--------+--------+
21471196| 10|
 1748515| 10|
 37146| 78|
18693412| 10|
 3979102| 10|
 807758| 10|
 6892592| 10|
 1539044| 10|
 1483633| 10|
 5087172| 10|
19308825| 10|
13494372| 10|
 1460361| 10|
14818872| 10|
22034166| 10|
 687322| 10|
18239213| 10|
19046161| 10|
19134120| 10|
 8392492| 10|
+--------+--------+
only showing top 20 rows

In [31]:
df = (df).join(min_w, 'id')
df = (df).join(max_w, 'id')
df = (df).join(avg_w, 'id')

In [32]:
df.count()

Out[ 25 ]: 11844

In [34]:
#for node in group_nodes.rdd.collect():
 #   print(node["id"])
 #   id = int(node["id"])
 #   tmp = g.edges.filter((col("src") == id) | (col("dst") == id))
 #   max = tmp.groupBy().max("weight").collect()
 #   min = tmp.groupBy().min("weight").collect()
 #   avg = tmp.groupBy().avg("weight").collect()
 #   df = df.withColumn("avg_weight", when(col("id")==id, avg[0][0]))
 #   df = df.withColumn("max_weight", when(col("id")==id, max[0][0]))      
 #   df = df.withColumn("min_weight", when(col("id")==id, min[0][0]))

In [35]:
df.show()

+--------+------+--------------------+-----------+-----------+----+-------------------+------+-------------------+--------------------+--------------------+--------+--------+------------------+
 id|degree| name|category_id|num_members|city| date_created|rating| avg_weight| min_weight| max_weight|sum(min)|sum(max)| sum(avg)|
+--------+------+--------------------+-----------+-----------+----+-------------------+------+-------------------+--------------------+--------------------+--------+--------+------------------+
 37146| 412|The Chicago Comic...| 29| 731| 2|2003-02-02 17:47:47| 4.6| 0.4785947396156348| 0.7261514341197416| 0.17675127021030212| 78| 202| 85.31143552311435|
 687322| 1513|Business is Art &...| 15| 1440| 1|2007-09-16 01:06:38| 4.73| 0.8561814161915097| 0.6013413189686693|0.030315296853126505| 10| 233|32.584097890641004|
 807758| 807|The Miramar Ski Club| 23| 1063| 1|2007-11-09 18:25:56| 4.8| 0.6600339368159014| 0.4733628738043564| 0.5763754122795762| 10| 217| 27.70616736977742|
 1288106| 336|SouthEastAsian Ex...| 10| 897| 3|2008-09-23 18:44:18| 4.79| 0.4598284741208738| 0.4287141361436222| 0.26349635957052775| 10| 100|21.805228758169932|
 1308839| 36|The San Francisco...| 26| 638| 3|2008-10-26 18:40:00| 4.82| 0.2136078061722022| 0.696185263777993| 0.8575079276444691| 11| 39|17.580952380952382|
 1460361| 1112|Heal the Loss of ...| 30| 1139| 1|2009-05-22 15:07:37| 4.44| 0.700516113844493| 0.3097309866417127| 0.8868040598700963| 10| 162|26.280244904285823|
 1483633| 1284| Clojure NYC| 34| 1457| 1|2009-07-03 18:50:28| 4.57| 0.9485132888563949| 0.9990353248297431| 0.7486740763499803| 10| 620| 50.81702217346944|
 1533340| 223|Hospitalented: Ne...| 10| 337| 1|2009-10-01 19:35:48| 4.5|0.48922983217420113| 0.5455162054280424| 0.1783522948862577| 10| 53|17.304740957966764|
 1539044| 689|Business Intellig...| 34| 793| 2|2009-10-12 19:02:57| 4.88| 0.8420770670692268| 0.9574456837207577| 0.37395590795634515| 10| 291| 38.60865084670232|
 1707405| 180| Automated Trading| 2| 125| 1|2010-09-15 19:02:04| 4.92| 0.5372999412775781| 0.30506806022424415| 0.7715282257952453| 10| 74|18.678260869565218|
 1716311| 37|Artists' Rights M...| 4| 107| 1|2010-10-03 06:36:11| 5.0| 0.5069749798969119| 0.1899904336519357| 0.34989508481677356| 10| 19|12.166666666666668|
 1748515| 1635| LispNYC| 34| 1548| 1|2010-12-09 14:23:53| 4.75|0.43469622885318515| 0.7361038684343991| 0.6527351578577881| 10| 646| 53.71283801748221|
 3979102| 1935| Angular-SF| 34| 4093| 3|2012-05-31 21:15:29| 4.64| 0.849269847876517|0.034334252696155865| 0.7490743192985286| 10| 2065| 85.75499848647317|
 5087172| 1565|New York Haskell ...| 34| 1830| 1|2012-09-24 21:26:02| 4.84|0.10601541987929353| 0.9114655235354971| 0.21430049845264143| 10| 570| 50.51599486521181|
 6892592| 1537|Singles Travel To...| 23| 6437| 2|2013-01-28 15:31:55| 0.0| 0.974245304810777| 0.9541166117523254| 0.2800499152439878| 10| 3186| 125.2949270203635|
 8392492| 2206|Manhattan Lecture...| 6| 1511| 1|2013-05-07 23:49:56| 4.85| 0.5339925617680185| 0.5888258974761916| 0.9520745413710369| 10| 407| 32.93963328168093|
 9265672| 353|Chicago Azure Meetup| 34| 241| 2|2013-07-08 15:54:35| 4.69| 0.346999328862611| 0.8368936401104013| 0.4555105915527484| 10| 177| 25.7722918675667|
11970832| 263|Hyde Park Runners...| 9| 467| 2|2014-01-09 19:00:13| 4.91| 0.1851875455173363| 0.5881887640148618| 0.5422232314936423| 10| 119|21.355461165048546|
12638262| 794|Tent & Trails Out...| 23| 1188| 1|2014-01-31 17:52:10| 4.85| 0.8762928741886573|0.033444054568777304| 0.22463115422371893| 10| 265|22.227520814061055|
12696752| 5|Ireland - A Hub f...| 34| 32| 3|2014-02-03 17:34:14| 0.0|0.18722265161089113| 0.12678080188191565| 0.6797307616617262| 11| 13| 12.25|
+--------+------+--------------------+-----------+-----------+----+-------------------+------+-------------------+--------------------+--------------------+--------+--------+------------------+
only showing top 20 rows

In [36]:
df_clean = df.na.drop()
df_clean.show()

+--------+------+--------------------+-----------+-----------+----+-------------------+------+--------+--------+------------------+
 id|degree| name|category_id|num_members|city| date_created|rating|sum(min)|sum(max)| sum(avg)|
+--------+------+--------------------+-----------+-----------+----+-------------------+------+--------+--------+------------------+
 37146| 412|The Chicago Comic...| 29| 731| 2|2003-02-02 17:47:47| 4.6| 78| 202| 85.31143552311435|
 687322| 1513|Business is Art &...| 15| 1440| 1|2007-09-16 01:06:38| 4.73| 10| 233|32.584097890641004|
 807758| 807|The Miramar Ski Club| 23| 1063| 1|2007-11-09 18:25:56| 4.8| 10| 217| 27.70616736977742|
 1288106| 336|SouthEastAsian Ex...| 10| 897| 3|2008-09-23 18:44:18| 4.79| 10| 100|21.805228758169932|
 1308839| 36|The San Francisco...| 26| 638| 3|2008-10-26 18:40:00| 4.82| 11| 39|17.580952380952382|
 1460361| 1112|Heal the Loss of ...| 30| 1139| 1|2009-05-22 15:07:37| 4.44| 10| 162|26.280244904285823|
 1483633| 1284| Clojure NYC| 34| 1457| 1|2009-07-03 18:50:28| 4.57| 10| 620| 50.81702217346944|
 1533340| 223|Hospitalented: Ne...| 10| 337| 1|2009-10-01 19:35:48| 4.5| 10| 53|17.304740957966764|
 1539044| 689|Business Intellig...| 34| 793| 2|2009-10-12 19:02:57| 4.88| 10| 291| 38.60865084670232|
 1707405| 180| Automated Trading| 2| 125| 1|2010-09-15 19:02:04| 4.92| 10| 74|18.678260869565218|
 1716311| 37|Artists' Rights M...| 4| 107| 1|2010-10-03 06:36:11| 5.0| 10| 19|12.166666666666668|
 1748515| 1635| LispNYC| 34| 1548| 1|2010-12-09 14:23:53| 4.75| 10| 646| 53.71283801748221|
 3979102| 1935| Angular-SF| 34| 4093| 3|2012-05-31 21:15:29| 4.64| 10| 2065| 85.75499848647317|
 5087172| 1565|New York Haskell ...| 34| 1830| 1|2012-09-24 21:26:02| 4.84| 10| 570| 50.51599486521181|
 6892592| 1537|Singles Travel To...| 23| 6437| 2|2013-01-28 15:31:55| 0.0| 10| 3186| 125.2949270203635|
 8392492| 2206|Manhattan Lecture...| 6| 1511| 1|2013-05-07 23:49:56| 4.85| 10| 407| 32.93963328168093|
 9265672| 353|Chicago Azure Meetup| 34| 241| 2|2013-07-08 15:54:35| 4.69| 10| 177| 25.7722918675667|
11970832| 263|Hyde Park Runners...| 9| 467| 2|2014-01-09 19:00:13| 4.91| 10| 119|21.355461165048546|
12638262| 794|Tent & Trails Out...| 23| 1188| 1|2014-01-31 17:52:10| 4.85| 10| 265|22.227520814061055|
12696752| 5|Ireland - A Hub f...| 34| 32| 3|2014-02-03 17:34:14| 0.0| 11| 13| 12.25|
+--------+------+--------------------+-----------+-----------+----+-------------------+------+--------+--------+------------------+
only showing top 20 rows

In [37]:
df_clean.count()

Out[ 44 ]: 11844

In [38]:
results = g.triangleCount()
triangle_count=results.select("id", "count")

In [39]:
df = (df).join(triangle_count, 'id')

In [40]:
oldColumns = df.schema.names
newColumns = ["id", "degree","name","category_id","num_members","city","date_created","rating","min_weight","max_weight","avg_weight", "triangles" ]

df = reduce(lambda data, idx: data.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), df)


In [41]:
display(df)

In [42]:
FEATURES_COL = ['degree','name','category_id', 'num_members', 'city', 'rating',  'min_weight', 'max_weight', 'avg_weight','triangles']

for col in df.columns:
    if col in FEATURES_COL:
        df = df.withColumn(col,df[col].cast('float'))
#df.show()
#df = df.na.drop()
vecAssembler = VectorAssembler(inputCols=FEATURES_COL, outputCol="features")
print(vecAssembler)
df_kmeans = vecAssembler.transform(df).select('id', 'features')
display(df_kmeans)

In [43]:
df.head(9)

In [44]:
df_kmeans.count()

Out[ 61 ]: 11844

In [45]:
cost = np.zeros(25)
for k in range(2,25):
    kmeans = KMeans().setK(k).setSeed(1).setFeaturesCol("features")
    model = kmeans.fit(df_kmeans.sample(False,0.1, seed=42))
    cost[k] = model.computeCost(df_kmeans) # requires Spark 2.0 or later
    print(k)
    print(cost[k])
    print('-------')

2
18069292800.7
-------
3
10491650034.1
-------
4
6697595070.64
-------
5
5735873914.0
-------
6
3694574014.08
-------
7
3003258105.15
-------
8
2682729661.7
-------
9
2433960803.18
-------
10
2478032822.11
-------
11
2169447618.21
-------
12
2303908858.14
-------
13
2042905472.41
-------
14
1964539564.52
-------
15
1900283046.52
-------
16
1806767448.86
-------
17
1794909820.67
-------
18
1769948540.2
-------
19
1764727304.58
-------
20
1679855686.24
-------
21
1622736318.06
-------
22
1622225341.13
-------
23
1557261466.19
-------
24
1568935177.68
-------

In [46]:
from pyspark.sql.types import *
display(spark.createDataFrame(cost[2:25].tolist(), FloatType()).toDF("x"))



x
1.80692931E10
1.049165E10
6.6975949E9
5.735874E9
3.69457408E9
3.00325811E9
2.68272973E9
2.4339607E9
2.4780329E9
2.16944768E9


In [47]:
# Look like there is very little gain after k=10, so we stick to that choice when processing the full data set

In [48]:
# Try out K Means clustering algorithm
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Trains a k-means model.
kmeans = KMeans().setK(10).setSeed(1)
model = kmeans.fit(df_kmeans)

# Make predictions
predictions = model.transform(df_kmeans)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Shows the result.
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

Silhouette with squared euclidean distance = 0.8110617232043946
Cluster Centers: 
[ 2.10417124e+01 5.60356751e+02 1.89023052e+00 4.01680571e+00
 5.15368277e+02 2.41533471e+01 1.00515917e+01 1.83752470e+02
 1.08409009e+05]
[ 2.22500000e+01 2.99033929e+03 1.11607143e+00 4.50941964e+00
 1.98748661e+03 4.94942484e+01 1.00223214e+01 8.73491071e+02
 9.05463000e+05]
[ 2.27554417e+01 1.77661588e+03 1.87580026e+00 4.44731114e+00
 1.12678361e+03 4.38102976e+01 1.00781050e+01 6.57048656e+02
 3.71002466e+05]
[ 1.94371257e+01 3.83989222e+03 1.00000000e+00 4.61497006e+00
 2.36310778e+03 5.70223375e+01 1.00479042e+01 1.11912575e+03
 1.12307608e+06]
[ 1.78983000e+01 1.76548969e+02 1.80923725e+00 2.52536784e+00
 1.15034903e+02 1.42784339e+01 9.19301941e+00 4.80690537e+01
 1.09996424e+04]
[ 2.41265823e+01 1.19071646e+04 1.00000000e+00 4.65316456e+00
 3.71860759e+03 1.07771722e+02 1.00759494e+01 2.71168354e+03
 1.62262211e+06]
[ 2.46238532e+01 5.79227523e+03 1.00000000e+00 4.65366975e+00
 2.86226606e+03 7.24163832e+01 1.00000000e+01 1.62368807e+03
 1.34978247e+06]
[ 2.37549296e+01 2.96619718e+03 1.57464789e+00 4.56988733e+00
 1.76217746e+03 5.30930865e+01 1.00563380e+01 9.07960563e+02
 7.06400392e+05]
[ 2.16183274e+01 1.03846174e+03 1.85409253e+00 4.33879003e+00
 8.19637011e+02 3.24847996e+01 1.00088968e+01 3.58884342e+02
 2.30040792e+05]
[ 2.25727612e+01 1.85527985e+03 1.69402985e+00 4.55061567e+00
 1.38984328e+03 4.09815785e+01 1.00167910e+01 5.60363806e+02
 5.36575424e+05]

In [49]:
predictions = predictions.join(df, 'id')
#display(predictions)

In [50]:
display(predictions.select("id","prediction", "degree","count"))

In [51]:
predictions.select("id","prediction", "degree","count").write.format("com.databricks.spark.csv").save("file:/databricks/driver/data.csv")

java.net.ConnectException: Connection refused
	at sun.nio.ch.SocketChannelImpl.checkConnect(Native Method)
	at sun.nio.ch.SocketChannelImpl.finishConnect(SocketChannelImpl.java:717)
	at sun.nio.ch.SocketAdaptor.connect(SocketAdaptor.java:111)
	at org.eclipse.jetty8.client.SelectConnector.startConnection(SelectConnector.java:82)
	at org.eclipse.jetty8.client.HttpDestination.startNewConnection(HttpDestination.java:292)
	at org.eclipse.jetty8.client.HttpDestination.doSend(HttpDestination.java:597)
	at org.eclipse.jetty8.client.HttpDestination.send(HttpDestination.java:532)
	at org.eclipse.jetty8.client.HttpClient.send(HttpClient.java:180)
	at com.databricks.rpc.JettyClient.$anonfun$sendRawExchange$2(JettyClient.scala:326)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:238)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:233)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:230)
	at com.databricks.rpc.JettyClient.withAttributionContext(JettyClient.scala:33)
	at com.databricks.rpc.JettyClient.sendRawExchange(JettyClient.scala:316)
	at com.databricks.rpc.JettyClient.sendExchange(JettyClient.scala:241)
	at com.databricks.rpc.JettyClient.sendAsync(JettyClient.scala:150)
	at com.databricks.rpc.BoundRPCClient.sendAsync(BoundRPCClient.scala:54)
	at com.databricks.backend.daemon.driver.DriverClient.com$databricks$backend$daemon$driver$DriverClient$$send(DriverClient.scala:95)
	at com.databricks.backend.daemon.driver.DriverClient.$anonfun$poll$1(DriverClient.scala:283)
	at com.databricks.util.FutureUtils$.retry(FutureUtils.scala:87)
	at com.databricks.util.FutureUtils$$anonfun$retry$2.applyOrElse(FutureUtils.scala:91)
	at com.databricks.util.FutureUtils$$anonfun$retry$2.applyOrElse(FutureUtils.scala:89)
	at scala.concurrent.Future.$anonfun$recoverWith$1(Future.scala:417)
	at scala.concurrent.impl.Promise.$anonfun$transformWith$1(Promise.scala:41)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at com.databricks.threading.NamedExecutor$$anon$2.$anonfun$run$1(NamedExecutor.scala:316)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:238)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:233)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:230)
	at com.databricks.threading.NamedExecutor.withAttributionContext(NamedExecutor.scala:257)
	at com.databricks.threading.NamedExecutor$$anon$2.run(NamedExecutor.scala:316)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)

In [52]:
# Typically at this point I would need to do something else with the data, which does not require Spark, so let's convert the Spark 
# dataframe to a good old Pandas dataframe for further processing.
import pandas as pd
pddf_pred = predictions.toPandas().set_index('id')
pddf_pred.head()

Out[ 35 ]: 
 features prediction \
id 
37146 [29.0, 731.0, 2.0, 4.59999990463, 412.0, 85.31... 0 
687322 [15.0, 1440.0, 1.0, 4.73000001907, 1513.0, 32.... 7 
807758 [23.0, 1063.0, 1.0, 4.80000019073, 807.0, 27.7... 8 
1288106 [10.0, 897.0, 3.0, 4.78999996185, 336.0, 21.80... 4 
1308839 [26.0, 638.0, 3.0, 4.82000017166, 36.0, 17.580... 4 

 degree name \
id 
37146 412.0 The Chicago Comic Book Meetup Group 
687322 1513.0 Business is Art & Art is Our Business Meetup 
807758 807.0 The Miramar Ski Club 
1288106 336.0 SouthEastAsian Expats in SF: Malaysia, Indones... 
1308839 36.0 The San Francisco Cavalier King Charles Spanie... 

 category_id num_members city date_created rating \
id 
37146 29.0 731.0 2.0 2003-02-02 17:47:47 4.60 
687322 15.0 1440.0 1.0 2007-09-16 01:06:38 4.73 
807758 23.0 1063.0 1.0 2007-11-09 18:25:56 4.80 
1288106 10.0 897.0 3.0 2008-09-23 18:44:18 4.79 
1308839 26.0 638.0 3.0 2008-10-26 18:40:00 4.82 

 avg_weight min_weight max_weight sum(min) sum(max) sum(avg) \
id 
37146 0.478595 0.726151 0.176751 78.0 202.0 85.311432 
687322 0.856181 0.601341 0.030315 10.0 233.0 32.584099 
807758 0.660034 0.473363 0.576375 10.0 217.0 27.706167 
1288106 0.459828 0.428714 0.263496 10.0 100.0 21.805229 
1308839 0.213608 0.696185 0.857508 11.0 39.0 17.580952 

 count 
id 
37146 68132.0 
687322 685177.0 
807758 262945.0 
1288106 52335.0 
1308839 573.0

In [53]:
pddf_pred.count()

In [54]:

threedee = plt.figure(figsize=(12,10)).gca(projection='3d')
threedee.scatter(pddf_pred.category_id, pddf_pred.num_members, pddf_pred.rating, c=pddf_pred.prediction)
threedee.set_xlabel('x')
threedee.set_ylabel('y')
threedee.set_zlabel('z')
display(plt.show())

In [55]:
from keras import optimizers
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors as nn
from itertools import islice
from matplotlib import pylab
import plotly.offline as plt
import plotly.graph_objs as go

tnse = TSNE(random_state=RS).fit_transform(df["features"])
np.set_printoptions(suppress=True)
Y = tsne.fit_transform(arr)
x_coords = Y[:, 0]
y_coords = Y[:, 1]
  
plot = [go.Scatter(x = x_coords,
                    y = y_coords,
                    mode = 'markers',
                    textposition='bottom center',
                    marker=dict(size=5,opacity=0.8))]

layout = go.Layout(title='Clustering')
fig = go.Figure(data=plot, layout=layout)
plt.iplot(fig)